In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/DataSet_Titanic.csv")
from sklearn.model_selection import train_test_split

data = df.drop(columns="Sobreviviente")
x_train, x_test, y_train, y_test = train_test_split(
    data, df["Sobreviviente"], test_size=0.2, random_state=123
)

random_forest_model: RandomForestClassifier = joblib.load(
    "D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

In [166]:
from sklearn.tree import DecisionTreeClassifier
from treeinterpreter import treeinterpreter as ti
import plotly.graph_objects as go
   

def getTreeInterpreterParamethers(instance, model: RandomForestClassifier | DecisionTreeClassifier, class_names):
    
    general_dict = {
            ("Instance", "Predictor"): [],
            ("Instance", "Value"): [],
        }               
    contribution_graph_data=[] 
    
    
        
    prediction, bias, contributions = ti.predict(model, instance)
    predictions_graph_data = {
        "labels": class_names,
        "values": prediction[0],
    }    

    for index, class_name in enumerate(class_names):
        contribution_graph_data.append({
                "class_name": class_name,
                "graph_data": []
            })
        general_dict[("Contribution" ,class_name)] = []
        bar_base = bias[0][index]
        media_array_x = ["Media Poblacional"]
        media_array_y = [bias[0][index]]
        sorted_contributions = sorted(
                zip(contributions[0], instance),
                key=lambda x: -max(x[0]),
            )
        for jIndex, (contribution, feature) in enumerate(sorted_contributions):
            media_array_x.append(feature)
            media_array_y.append(bar_base)
            if feature not in general_dict[("Instance", "Predictor")]:
                general_dict[("Instance", "Predictor")].append(feature)
                general_dict[("Instance", "Value")].append(pd.Series(instance[feature]).values[0])
            general_dict[("Contribution" ,class_name)].append(contribution[index])    
            x = [feature]
            y = [feature]
            for sorted_contribution in sorted_contributions[jIndex:]: 
                x.append(sorted_contribution[1])          
                y.append(contribution[index])          
            contribution_graph_data[index]["graph_data"].append(go.Bar(name=feature, x=x, y=y)) 
        general_dict[("Contribution" ,class_name)].append(bias[0][index]) 
        contribution_graph_data[index]["graph_data"].insert(0, go.Bar(name='Media', x=media_array_x, y=media_array_y))
        contribution_graph_data[index]["graph_data"].append(go.Bar(name='Prediction', x=["Predicción Final"], y=[prediction[0][index]]))
    
    general_dict[("Instance", "Predictor")].append("Media Poblacional")
    general_dict[("Instance", "Value")].append("-")  
    return contribution_graph_data, general_dict, predictions_graph_data

In [167]:
n=40
if n >= 1 and n <= len(x_test):
    print("in")
    instance: pd.DataFrame = x_test[n-1:n] 

    contribution_graph_data, general_dict, predictions_graph_data = getTreeInterpreterParamethers(
        instance=instance,
        class_names=["m", "v"],
        model=random_forest_model,
    )



in


d:\CARRERA\TESIS\Flask_template_auth_with_Dash\virtual_env\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but DecisionTreeClassifier was fitted without feature names

d:\CARRERA\TESIS\Flask_template_auth_with_Dash\virtual_env\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but DecisionTreeClassifier was fitted without feature names

d:\CARRERA\TESIS\Flask_template_auth_with_Dash\virtual_env\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but DecisionTreeClassifier was fitted without feature names

d:\CARRERA\TESIS\Flask_template_auth_with_Dash\virtual_env\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but DecisionTreeClassifier was fitted without feature names

d:\CARRERA\TESIS\Flask_template_auth_with_Dash\virtual_env\Lib\site-packages\sklearn\base.py:486: UserWarning:

X has feature names, but DecisionTreeClassifier was fitted without feature names

d:\CARRERA\TESIS\Flask_templat

In [168]:
df_general: pd.DataFrame = pd.DataFrame(general_dict)

In [169]:
df_general

Instance       Contribution          
  Predictor Value            m         v
0    Genero     0     0.137088 -0.137088
1    PadHij     0     0.018124 -0.018124
2     Clase     2    -0.007804  0.007804
3      Edad  30.0     0.007718 -0.007718
4   HermEsp     1    -0.005523  0.005523
5     Media     -     0.589597  0.410403

In [150]:
import plotly.graph_objects as go
import plotly.express as px

In [173]:
for data in contribution_graph_data:
    fig = go.Figure(data["graph_data"], layout=dict(barmode='stack'))
    # Change the bar mode
    # fig.update_layout(barmode='stack')
    fig.show()

In [171]:

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=predictions_graph_data["labels"], values=predictions_graph_data["values"], hole=.3)])
fig.show()